# k-Means
Der k-Means-Algorithmus ist eines der beliebtesten Verfahren zur Clusteranalyse. Es lässt sich sehr einfach implementieren und findet schnell entsprechende Clusterzentren.

In [ ]:
import numpy as np

from tui_dsmt.clustering import animate_kmeans, interactive_kmeans
from tui_dsmt.clustering.datasets import clustering_example1, clustering_example2

## Inhaltsverzeichnis
- [Der Algorithmus](#Der-Algorithmus)
- [Visualisierung](#Visualisierung)
- [Erweiterte Implementierungen](#Erweiterte-Implementierungen)
- [Probleme](#Probleme)

## Der Algorithmus
k-Means verwendet sogenannte Zentroide, also künstliche Punkte, die als Repräsentanten für jeweils einen Cluster gelten. Das Ziel ist, eine feste Anzahl dieser Zentroide genau so zu verteilen, dass die Varianz innerhalb der Cluster möglichst gering wird. Es ist also ein Abstandsmaß notwendig, wobei im Folgenden die euklidische Distanz zur Anwendung kommt.

Der Algorithmus verläuft wie folgt:
1. Wähle $k$ Clusterzentren zufällig aus dem Datensatz. Die Anzahl $k$ muss also vorher bekannt sein.
2. wiederhole bis zur Konvergenz:
  1. Jeder Punkt wird dem Cluster zugeordnet, dessen Zentrum er am nächsten liegt.
  2. Jedes Clusterzentrum wird als Mittelpunkt aller zugehörigen Punkte neu bestimmt.

Die folgende Funktion arbeitet als Generator und gibt immer wieder neue Clusterzentren zurück, bis sie nicht mehr aufgerufen wird. Die Prüfung des Abbruchkriteriums fällt damit in die Verantwortung des aufrufenden Programmbestandteils.

In [ ]:
def k_means(xs, ys, k):
    # Die Clusterzentren werden zufällig gewählt.
    idx = np.random.choice(np.arange(len(xs)), k, replace=False)

    centroids_x = xs[idx]
    centroids_y = ys[idx]

    yield centroids_x, centroids_y

    # Die Clusterzentren werden wiederholt neu bestimmt.
    while True:
        # Die Variablen dienen dem Speichern der neuen
        # Position der Clusterzentren.
        new_centroids_x = np.zeros(k)
        new_centroids_y = np.zeros(k)

        centroids_len = np.zeros(k)

        # Die Clusterzugehörigkeit wird für alle Punkte
        # bestimmt.
        for x, y in zip(xs, ys):
            # Dazu wird die Distanz zu jedem Cluster-
            # mittelpunkt berechnet. Das Minimum
            # beschreibt den nahegelegendsten Cluster.
            distances = np.sqrt((centroids_x - x) ** 2 + (centroids_y - y) ** 2)
            centroid_index = distances.argmin()

            # Die Koordinaten des Punktes werden zur
            # Neubestimmung des Clusterzentrums
            # gespeichert.
            new_centroids_x[centroid_index] += x
            new_centroids_y[centroid_index] += y

            centroids_len[centroid_index] += 1

        # Clustermittelpunkte, denen keine Punkte zu-
        # geordnet wurden, werden erneut zufällig
        # verteilt.
        for i in range(len(centroids_len)):
            if centroids_len[i] == 0:
                new_centroids_x[i] = np.random.uniform(xs.min(), xs.max())
                new_centroids_y[i] = np.random.uniform(ys.min(), ys.max())
                centroids_len[i] = 1

        # Die summierten Punkte werden durch die Anzahl
        # geteilt, um neue Clustermittelpunkte zu
        # erhalten.
        centroids_x = new_centroids_x / centroids_len
        centroids_y = new_centroids_y / centroids_len

        yield centroids_x, centroids_y

## Visualisierung

In [ ]:
animate_kmeans(k_means, clustering_example1, k=15)

## Erweiterte Implementierungen
Wie Sie gesehen haben, hängt der Algorithmus von der Initialisierung des Zufallszahlengenerators ab. Sie können daher beobachten, dass einige der Häufungen im äußeren Bereich aufgeteilt werden, während sich übergreifende Cluster in der Mitte bilden. Es kann sich daher lohnen, die Methode mehrfach auszuführen und das beste Ergebnis auszuwählen. Eine fertige Implementierung, die den k-Means-Algorithmus mehrfach ausführt und weitere Verbesserungen enthält, entstammt dem Paket `sklearn`.

Initialisieren Sie dazu ein Objekt der Klasse `KMeans` und übergeben Sie dem Parameter die Anzahl der Clusterzentren. Rufen Sie anschließend `fit_predict` mit einem DataFrame auf, das die Angaben zu den entsprechenden Dimensionen enthält. Zurückgegeben wird ein Array aus Clusterindizes.

In [ ]:
from sklearn.cluster import KMeans

k = 15
KMeans(k).fit_predict(clustering_example1[['x', 'y']])

In der nachfolgenden Zelle haben Sie die Möglichkeit mit dem Parameter $k$ zu experimentieren.

In [ ]:
interactive_kmeans(clustering_example1)

## Probleme
Der k-Means Algorithmus eignet sich jedoch nicht für alle Daten. Mengen, die nicht konvex sind, können beispielsweise nicht korrekt in Cluster eingeteilt werden. Die ineinandergreifenden Spiralen demonstrieren anschaulich die Grenzen des Algorithmus.

In [ ]:
animate_kmeans(k_means, clustering_example2, k=3)